In [10]:
!pip install langgraph

In [2]:
!pip install langchain langchain-community langchain-openai langchain_experimental langchain_groq  -q

In [11]:
from langchain.tools import Tool
from langchain_experimental.tools import PythonREPLTool
from langchain.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType
#from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import os
import re


In [ ]:
def add(kwargs):
    """Adds two numbers."""
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 + num2
    except Exception as e:
        return f"Error: {str(e)}"

def multiply(kwargs):
    """Multiplies two numbers."""
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 * num2
    except Exception as e:
        return f"Error: {str(e)}"

def divide(kwargs):
    """Divides two numbers."""
    try:
        num1, num2 = float(kwargs["num1"]), float(kwargs["num2"])
        return num1 / num2 if num2 != 0 else "Cannot divide by zero"
    except Exception as e:
        return f"Error: {str(e)}"

def subtract(kwargs):
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 - num2
    except Exception as e:
        return f"Error: {str(e)}"

def power(kwargs):
    """Raises the first number to the power of the second number."""
    try:
        num1, num2 = int(kwargs["num1"]), int(kwargs["num2"])
        return num1 ** num2
    except Exception as e:
        return f"Error: {str(e)}"

def summarize(kwargs):
    """Summarizes the given text."""
    return " ".join(kwargs.split()[:10]) + "..."

def word_count(kwargs):
    """Counts the number of words in the given text."""
    return len(kwargs.split(' '))


In [ ]:
add_tool = Tool(name="Addition", func=add, description="Adds two numbers.")
multiply_tool = Tool(name="Multiplication", func=multiply, description="Multiplies two numbers.")
divide_tool = Tool(name="Division", func=divide, description="Divides two numbers.")
subtract_tool = Tool(name="Subtraction", func=subtract, description="Subtracts two numbers.")
power_tool = Tool(name="Power", func=power, description="Raises the first number to the power of the second number.")
summarize_tool = Tool(name="Summarization", func=summarize, description="Summarizes a given text.")
word_count_tool = Tool(name="WordCount", func=word_count, description="Counts words in a given text.")

In [ ]:
tools = [add_tool, multiply_tool, divide_tool, subtract_tool,power_tool,summarize_tool,word_count_tool]

llm = ChatGroq(model_name="gemma2-9b-it", temperature=0,api_key = "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")

In [12]:
memory = MemorySaver()

In [13]:
agent = create_react_agent(llm, tools, checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p7y0', 'function': {'arguments': '{"__arg1":"hi im bob! and i live in sf"}', 'name': 'Summarization'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1628, 'total_tokens': 1718, 'completion_time': 0.163636364, 'prompt_time': 0.052439326, 'queue_time': 0.021984560999999993, 'total_time': 0.21607569}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c59a29ea-cc82-446f-a052-0f6724cf8c69-0', tool_calls=[{'name': 'Summarization', 'args': {'__arg1': 'hi im bob! and i live in sf'}, 'id': 'call_p7y0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1628, 'output_tokens': 90, 'total_tokens': 1718})]}}
----
{'tools': {'messages': [ToolMessage(content='hi im bob! and i live in sf...', name='Summarization', id='e449b82b-82b4-4aa0-bc6c-f880ec7d992a', tool_call_id='call_p7

In [ ]:
# response = agent.run("Do you know what my name is?")
# print(response)

In [15]:
for chunk in agent.stream(
    {"messages": [HumanMessage(content="Do you know my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Yes, you told me your name is Bob.  ', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 1757, 'total_tokens': 1770, 'completion_time': 0.023636364, 'prompt_time': 0.056908694, 'queue_time': 0.024892145999999997, 'total_time': 0.080545058}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-429a1524-e147-46af-ad7d-c83adc2fffa6-0', usage_metadata={'input_tokens': 1757, 'output_tokens': 13, 'total_tokens': 1770})]}}
----


In [16]:
for chunk in agent.stream(
    {"messages": [HumanMessage(content="Tell me my name")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1782, 'total_tokens': 1791, 'completion_time': 0.016363636, 'prompt_time': 0.081526482, 'queue_time': 0.10872424700000001, 'total_time': 0.097890118}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d348a206-414e-42f0-994d-d4dfcbda456d-0', usage_metadata={'input_tokens': 1782, 'output_tokens': 9, 'total_tokens': 1791})]}}
----


In [17]:
for chunk in agent.stream(
    {"messages": [HumanMessage(content="Count word from i love bangladesh so much")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nhm9', 'function': {'arguments': '{"__arg1":"i love bangladesh so much"}', 'name': 'WordCount'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1808, 'total_tokens': 1861, 'completion_time': 0.096363636, 'prompt_time': 0.058417711, 'queue_time': 0.023738401000000006, 'total_time': 0.154781347}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3929e020-075c-4acd-b7b4-f50a79e968b0-0', tool_calls=[{'name': 'WordCount', 'args': {'__arg1': 'i love bangladesh so much'}, 'id': 'call_nhm9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1808, 'output_tokens': 53, 'total_tokens': 1861})]}}
----
{'tools': {'messages': [ToolMessage(content='5', name='WordCount', id='072f4ff8-90d7-4840-86db-cf8c7b82ce79', tool_call_id='call_nhm9')]}}
----
{'agent': {'messages': [AIMessa